In [1]:
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
import cv2
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

# Akses environment
DATA_PATH = os.getenv('DATA_PATH')

In [3]:
imgs = []
labels = []

for folder in os.listdir(DATA_PATH):
    for i, img in enumerate(os.listdir(os.path.join(DATA_PATH, folder))):
        img = cv2.imread(os.path.join(DATA_PATH, folder, img))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgs.append(gray)
        labels.append(folder)

In [4]:
print('Jumlah gambar: ', len(imgs))
print('Label: ', len(labels))

Jumlah gambar:  12446
Label:  12446


In [5]:
def calc_glcm_all_agls(img, label, props, distances=[1,2,3,4,5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    glcm = graycomatrix(img,
                        distances=distances,
                        angles=agls,
                        levels=lvl,
                        symmetric=sym,
                        normed=norm)
    feature = []
    for dist in range(0, len(distances)):
        glcm_props = [propery for name in props for propery in graycoprops(glcm, name)[dist]]
        for item in glcm_props:
            feature.append(item)
            
    feature.append(label)
    return feature

In [6]:
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels):
    glcm_all_agls.append(
        calc_glcm_all_agls(
            img,
            label,
            props=properties
        )
    )

KeyboardInterrupt: 

In [60]:
columns = []
angles = ['0', '45', '90', '135']
distances = [1, 2, 3, 4, 5]

for distance in distances:
    for name in properties:
        for ang in angles:
            columns.append(f"{name}_{distance}_{ang}")

columns.append("label")

In [61]:
import pandas as pd

glcm_df = pd.DataFrame(glcm_all_agls,
                        columns = columns)

# Save to csv
glcm_df.to_csv("glcm_kidney_illness_extended_dists.csv")